# Topic 2: Recommendation System
### Demo: Underthesea, Gemsim, Cosin Similarity

In [1]:
!pip install gensim
!pip install underthesea

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re

In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/MyDrive/LDS0/Topic_12_Hasaki/demo'

Mounted at /content/gdrive
/content/gdrive/MyDrive/LDS0/Topic_12_Hasaki/demo


In [4]:
STOP_WORD_FILE = 'files/vietnamese-stopwords.txt'

In [5]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

# Underthesea
#### word_tokenize, pos_tag, sent_tokenize
- link: https://github.com/undertheseanlp/underthesea

In [6]:
# Dữ liệu thử nghiệm
df= pd.read_csv('San_pham_temp.csv')

In [7]:
df.head()

,ma_san_pham,ten_san_pham,mo_ta
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,Nước Hoa Hồng Klairs Supple Preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...


In [8]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 1 cột, và chỉ lấy 200 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['mo_ta'].apply(lambda x: ' '.join(x.split()[:200]))

In [9]:
df.head()

,ma_san_pham,ten_san_pham,mo_ta,Content
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa Hồng Klairs Supple Preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...


In [10]:
# word_tokenize
df["Content_wt"]=df["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [11]:
df[["Content", "Content_wt"]].head(2)

,Content,Content_wt
0,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
1,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy_Trang L'Oréal là dòng sản_phẩm tẩy_tr...


In [12]:
import re

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.072939,0.087811,0.054862,0.094169,0.102876,0.038985,0.102876,0.031367,0.072939,0.087787,0.117037,0.087856,0.049119,0.112244
1,0.072939,1.000000,0.064760,0.350432,0.526668,0.374571,0.616630,0.374571,0.035444,1.000000,0.275089,0.377327,0.215550,0.724976,0.715834
2,0.087811,0.064760,1.000000,0.090247,0.060034,0.116160,0.048472,0.116160,0.064192,0.064760,0.073337,0.162889,0.068077,0.069803,0.075385
3,0.054862,0.350432,0.090247,1.000000,0.489055,0.201800,0.412685,0.201800,0.263826,0.350432,0.292934,0.220773,0.324831,0.327950,0.335600
4,0.094169,0.526668,0.060034,0.489055,1.000000,0.263631,0.581359,0.263631,0.227203,0.526668,0.386242,0.350573,0.383020,0.461990,0.513634
5,0.102876,0.374571,0.116160,0.201800,0.263631,1.000000,0.253279,1.000000,0.072038,0.374571,0.209241,0.217873,0.125903,0.353338,0.383388
6,0.038985,0.616630,0.048472,0.412685,0.581359,0.253279,1.000000,0.253279,0.188687,0.616630,0.485731,0.340624,0.296315,0.552316,0.557042
7,0.102876,0.374571,0.116160,0.201800,0.263631,1.000000,0.253279,1.000000,0.072038,0.374571,0.209241,0.217873,0.125903,0.353338,0.383388
8,0.031367,0.035444,0.064192,0.263826,0.227203,0.072038,0.188687,0.072038,1.000000,0.035444,0.234050,0.042711,0.289650,0.022963,0.025429
9,0.072939,1.000000,0.064760,0.350432,0.526668,0.374571,0.616630,0.374571,0.035444,1.000000,0.275089,0.377327,0.215550,0.724976,0.715834


In [15]:
# Hàm đề xuất sản phẩm
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations_(ma_san_pham, cosine_sim=cosine_sim, nums=3):
    idx = df.index[df['ma_san_pham'] == ma_san_pham][0]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy 3 sản phẩm tương tự nhất
    hotel_indices = [i[0] for i in sim_scores]
    return df.iloc[hotel_indices]

In [16]:
# Gọi function
recommendations = get_recommendations_(318900012)
recommendations

0


,ma_san_pham,ten_san_pham,mo_ta,Content,Content_wt
11,422204795,"Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt_Cosrx Tràm_Trà , 0.5 % BHA Có Độ_p..."
14,422204884,Combo 2 Nước Tẩy Trang Bí Đao Cocoon Làm Sạch ...,Nước Tẩy Trang Bí Đao Cocoon Winter Melon Mice...,Nước Tẩy Trang Bí Đao Cocoon Winter Melon Mice...,Nước Tẩy_Trang Bí_Đao Cocoon_Winter Melon_Mice...
5,200400003,Nước Tẩy Trang Bioderma Dành Cho Da Nhạy Cảm 5...,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,Nước Tẩy_Trang Bioderma bao_gồm 2 phân_loại : ...


#### Lưu ma trận kết quả consin và đọc lên khi cần đề xuất

In [17]:
# Save cosine_sim to file
import pickle
with open('products_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('products_cosine_sim.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [18]:
def get_recommendations(df, ma_san_pham, cosine_sim=cosine_sim, nums=5):
    # Get the index of the hotel that matches the hotel_id
    matching_indices = df.index[df['ma_san_pham'] == ma_san_pham].tolist()
    if not matching_indices:
        print(f"No hotel found with ID: {ma_san_pham}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]

    # Get the pairwise similarity scores of all hotels with that hotel
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the hotels based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar hotels (Ignoring the hotel itself)
    sim_scores = sim_scores[1:nums+1]

    # Get the hotel indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top n most similar hotels as a DataFrame
    return df.iloc[product_indices]

In [19]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df, 318900012, cosine_sim=cosine_sim_new, nums=3)
recommendations

,ma_san_pham,ten_san_pham,mo_ta,Content,Content_wt
11,422204795,"Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt Cosrx Tràm Trà, 0.5% BHA Có Độ pH ...","Gel Rửa Mặt_Cosrx Tràm_Trà , 0.5 % BHA Có Độ_p..."
14,422204884,Combo 2 Nước Tẩy Trang Bí Đao Cocoon Làm Sạch ...,Nước Tẩy Trang Bí Đao Cocoon Winter Melon Mice...,Nước Tẩy Trang Bí Đao Cocoon Winter Melon Mice...,Nước Tẩy_Trang Bí_Đao Cocoon_Winter Melon_Mice...
5,200400003,Nước Tẩy Trang Bioderma Dành Cho Da Nhạy Cảm 5...,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,Nước Tẩy_Trang Bioderma bao_gồm 2 phân_loại : ...
